In [14]:
import pandas as pd

In [15]:
eng_seg = pd.read_parquet("../dataset/train_df_English_translated.parquet")
fre_seg = pd.read_parquet("../dataset/train_df_French_translated.parquet")
kor_seg = pd.read_parquet("../dataset/train_df_Korean_translated.parquet")
chn_esg = pd.read_parquet("../dataset/train_df_Chinese_translated.parquet")

In [16]:
eng_seg = eng_seg.drop_duplicates()
fre_seg = fre_seg.drop_duplicates()
kor_seg = kor_seg.drop_duplicates()
chn_esg = chn_esg.drop_duplicates()

In [17]:
# Korean Dataset Processing
def aggregator(df):
    df_agg = df.groupby("title").agg({"impact_length_idx":"first",'sentence':"sum","language":"first","content":"sum"})
    df_agg["title_info"] = df_agg.index
    df_agg.reset_index(inplace=True)
    df_agg["sep"] = " || "
    df_agg["feature"] = df_agg["title_info"] + df_agg["sep"] + df_agg["content"]
    df_agg["label"] = df_agg["impact_length_idx"]
    df_agg = df_agg[["title_info", "language", "feature", "label"]]
    return df_agg

### Korean Dataset Aggregation

In [18]:
# Separate the title & Body
kor_seg["title_eng"] = kor_seg["Translation"].apply(lambda x : x.split("||")[0].strip())
kor_seg["body"] = kor_seg["Translation"].apply(lambda x : x.split("||")[-1].strip())
# Perform Aggregations
df_agg = kor_seg.groupby("title").agg({"impact_length_idx":"first",'sentence':"sum","language":"first","content":"sum","title_eng":"first","body":"sum"})
df_agg["title_info"] = df_agg.index
df_agg.reset_index(inplace=True)
df_agg["sep"] = " || "
df_agg["feature"] = df_agg["title_eng"] + df_agg["sep"] + df_agg["body"]
df_agg["label"] = df_agg["impact_length_idx"]
df_agg = df_agg[["title_info", "language", "feature", "label"]]
kor_seg_agg = df_agg

### Chinese Dataset Aggregation

In [19]:
chn_esg["title_info"] = chn_esg["title"]
chn_esg["feature"] = chn_esg["Translation"]
chn_esg["label"] = chn_esg["impact_length_idx"]
chn_seg_agg = chn_esg[["title_info", "language", "feature", "label"]]

### French Dataset Aggregation

In [20]:
# Perform Aggregations
fre_seg["sep"] = fre_seg["Translation"].apply(lambda x : x.split("||"))
fre_seg["title_eng"] = fre_seg["sep"].apply(lambda x : x[0].strip() if len(x) == 2 else "")
fre_seg["body"] = fre_seg["sep"].apply(lambda x : x[1].strip() if len(x) == 2 else x[0].strip())

# Perform Aggregations
df_agg = fre_seg.groupby("title").agg({"impact_length_idx":"first",'sentence':"sum","language":"first","content":"sum","title_eng":"first","body":"sum"})
df_agg["title_info"] = df_agg.index
df_agg.reset_index(inplace=True)
df_agg["sep"] = " || "
df_agg["feature"] = df_agg["title_eng"] + df_agg["sep"] + df_agg["body"]
df_agg["label"] = df_agg["impact_length_idx"]
df_agg = df_agg[["title_info", "language", "feature", "label"]]
fre_seg_agg = df_agg

### English Dataset Aggregation

In [21]:
df_agg = eng_seg.groupby("title").agg({"impact_length_idx":"first",'sentence':"sum","language":"first","content":"sum"})
df_agg["title_info"] = df_agg.index
df_agg.reset_index(inplace=True)
df_agg["sep"] = " || "
df_agg["feature"] = df_agg["title_info"] + df_agg["sep"] + df_agg["content"]
df_agg["label"] = df_agg["impact_length_idx"]
df_agg = df_agg[["title_info", "language", "feature", "label"]]
eng_seg_agg = df_agg

In [22]:
df = pd.concat([eng_seg_agg, fre_seg_agg, kor_seg_agg, chn_seg_agg])

In [23]:
def word_count(text):
    return len(text.split(" "))

In [24]:
df["word_count"] = df["feature"].map(word_count)
df["source"] = "normal"

In [25]:
df.to_parquet("../dataset/consolidated.parquet")

In [26]:
df

,title_info,language,feature,label,word_count,source
0,$11.4T Investor Coalition Pressures Fast Food ...,English,$11.4T Investor Coalition Pressures Fast Food ...,2.0,60,normal
1,100 EU Cities Commit to Reach Climate Neutrali...,English,100 EU Cities Commit to Reach Climate Neutrali...,2.0,124,normal
2,3M to Invest $1 Billion Towards Achieving New ...,English,3M to Invest $1 Billion Towards Achieving New ...,2.0,146,normal
3,"Accenture Announces New Sustainability Goals, ...",English,"Accenture Announces New Sustainability Goals, ...",2.0,29,normal
4,Accenture Continues Series of ESG Acquisitions...,English,Accenture Continues Series of ESG Acquisitions...,0.0,40,normal
...,...,...,...,...,...,...
892,每年只開7個月，竟能賺進10億元！這家百年老店取螯不殺蟹，用「永續思維」和「賠錢心態」賣出長...,Chinese,"Open only 7 months a year, but can earn into 1...",2.0,1392,normal
893,中油首批碳中和原油抵台，可達88座大安森林公園1年吸碳量,Chinese,CNPC's first batch of carbon-neutral crude oil...,1.0,772,normal
894,長榮經營權兄弟相爭》殷琪手握長榮鋼關鍵票數「會支持最維護股東權益的那一方」,Chinese,Evergreen's management rights brothers fight e...,0.0,760,normal
895,張國煒寵員工無極限！星宇9月「調薪5％」獲證實 邊境鬆綁明年能獲利？他曝「這時機」才行,Chinese,Zhang Guowei favors his employees to no end! S...,0.0,572,normal
